In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [37]:
dataset = pd.read_csv("international-airline-passengers.csv", usecols=[1], engine='python', skipfooter=3)
dataset = dataset.values.astype('float32') # convert the dataframe values to a numpy array
print(type(dataset))
plt.plot(dataset)
plt.show()

<class 'numpy.ndarray'>


In [17]:
# fix random seed for reproducibility
np.random.seed(7)

In [21]:
# normalize the dataset because the tanh and sigmoid functions are sensitive to the scale of the data. If the values are between 0-1 they work well
scaler = MinMaxScaler(feature_range=(0,1))
dataset = scaler.fit_transform(dataset)
print(dataset)

[[0.01544401]
 [0.02702703]
 [0.05405405]
 [0.04826255]
 [0.03281853]
 [0.05984557]
 [0.08494207]
 [0.08494207]
 [0.06177607]
 [0.02895753]
 [0.        ]
 [0.02702703]
 [0.02123553]
 [0.04247104]
 [0.07142857]
 [0.05984557]
 [0.04054055]
 [0.08687258]
 [0.12741312]
 [0.12741312]
 [0.10424709]
 [0.05598456]
 [0.01930502]
 [0.06949806]
 [0.07915059]
 [0.08880308]
 [0.14285713]
 [0.11389962]
 [0.13127413]
 [0.14285713]
 [0.18339768]
 [0.18339768]
 [0.15444016]
 [0.11196911]
 [0.08108109]
 [0.1196911 ]
 [0.12934363]
 [0.14671814]
 [0.17181468]
 [0.14864865]
 [0.15250966]
 [0.22007722]
 [0.24324325]
 [0.26640925]
 [0.2027027 ]
 [0.16795367]
 [0.13127413]
 [0.17374519]
 [0.17760617]
 [0.17760617]
 [0.25482625]
 [0.25289574]
 [0.24131274]
 [0.26833975]
 [0.3088803 ]
 [0.32432434]
 [0.25675675]
 [0.20656371]
 [0.14671814]
 [0.18725869]
 [0.19305018]
 [0.16216215]
 [0.25289574]
 [0.23745173]
 [0.25096524]
 [0.3088803 ]
 [0.38223937]
 [0.36486486]
 [0.2992278 ]
 [0.24131274]
 [0.1911197 ]
 [0.24

In [32]:
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size], dataset[train_size:]

In [44]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
        
    print(dataX)
    #print(dataY)
    return np.array(dataX), np.array(dataY)


In [45]:
create_dataset(dataset, 1)

[array([112.], dtype=float32), array([118.], dtype=float32), array([132.], dtype=float32), array([129.], dtype=float32), array([121.], dtype=float32), array([135.], dtype=float32), array([148.], dtype=float32), array([148.], dtype=float32), array([136.], dtype=float32), array([119.], dtype=float32), array([104.], dtype=float32), array([118.], dtype=float32), array([115.], dtype=float32), array([126.], dtype=float32), array([141.], dtype=float32), array([135.], dtype=float32), array([125.], dtype=float32), array([149.], dtype=float32), array([170.], dtype=float32), array([170.], dtype=float32), array([158.], dtype=float32), array([133.], dtype=float32), array([114.], dtype=float32), array([140.], dtype=float32), array([145.], dtype=float32), array([150.], dtype=float32), array([178.], dtype=float32), array([163.], dtype=float32), array([172.], dtype=float32), array([178.], dtype=float32), array([199.], dtype=float32), array([199.], dtype=float32), array([184.], dtype=float32), array([16

(array([[112.],
        [118.],
        [132.],
        [129.],
        [121.],
        [135.],
        [148.],
        [148.],
        [136.],
        [119.],
        [104.],
        [118.],
        [115.],
        [126.],
        [141.],
        [135.],
        [125.],
        [149.],
        [170.],
        [170.],
        [158.],
        [133.],
        [114.],
        [140.],
        [145.],
        [150.],
        [178.],
        [163.],
        [172.],
        [178.],
        [199.],
        [199.],
        [184.],
        [162.],
        [146.],
        [166.],
        [171.],
        [180.],
        [193.],
        [181.],
        [183.],
        [218.],
        [230.],
        [242.],
        [209.],
        [191.],
        [172.],
        [194.],
        [196.],
        [196.],
        [236.],
        [235.],
        [229.],
        [243.],
        [264.],
        [272.],
        [237.],
        [211.],
        [180.],
        [201.],
        [204.],
        [188.],
        